In [ ]:
import awswrangler as wr
import pandas as pd
import os

AMBIENTE = os.getenv("AMBIENTE")

AMBIENTE

In [ ]:
roles = wr.s3.read_parquet(f"s3://cchc-dw-{AMBIENTE}-staging/conecta/roles",columns=['per_id','numero_socio','id_rol','estado_relacion'])
ficha_personas = wr.s3.read_parquet(f"s3://cchc-dw-{AMBIENTE}-staging/conecta/ficha_personas/estado=ACTIVO/",columns=['per_id','rut','nombre_completo','telefono_comercial','telefono_movil_comercial','email_comercial'])

ficha_personas.head()

In [ ]:
df_roles = roles.copy()
df_ficha_personas = ficha_personas.copy()

df_roles = df_roles[(df_roles.id_rol == 106) & (df_roles.estado_relacion == 'ACTIVA')]

df_merge = df_roles.merge(df_ficha_personas, how='left', on='per_id')

df_merge.rename(columns={
    "per_id": "id_persona",
    "numero_socio": "id_socio",
    "nombre_completo": "nombre",
    "telefono_movil_comercial": "celular_comercial",
    "email_comercial": "mail_comercial"
},inplace=True)

df_merge = df_merge[['id_socio', 'id_persona', 'rut', 'nombre', 'telefono_comercial', 'celular_comercial', 'mail_comercial']]


wr.s3.to_parquet(
        df=df_merge,
        path=f"s3://cchc-dw-{AMBIENTE}-analytics/dim_representantes_camara",
        dataset=True,
        mode="overwrite",
)